<a href="https://colab.research.google.com/github/Toan02Ky-UIT/CodeProject/blob/main/model_dau_tien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/gdrive/MyDrive/Project/Bai2/annonimized.csv')


In [ ]:
df = df.drop(columns=["concat('it001',`language_id`)", 'updated_at'])

In [ ]:
df = df.rename(columns={
    "concat('it001',`assignment_id`)": "assignment_id",
    "concat('it001',`problem_id`)": "problem_id",
    "concat('it001', username)": "username"
})


In [ ]:
total_per_user = df.loc[df['is_final'] == 1].copy() #Có copy thì có bớt đi ít lỗi khó hiểu, cần tìm hiểu thêm
total_per_user['total_coefficient'] = total_per_user['coefficient'] * total_per_user['pre_score']

total_per_user = total_per_user.groupby('username', as_index=False)['total_coefficient'].sum()

total_per_user.rename(columns={'coefficient': 'total_coefficient'}, inplace=True)

In [ ]:
assignment_count = df.groupby('username')['assignment_id'].nunique().reset_index()
assignment_count.rename(columns={'assignment_id': 'num_assignments_submitted'}, inplace=True)

In [ ]:
submission_count = df.groupby('username', as_index=False)['problem_id'].count()
submission_count = submission_count.rename(columns={'problem_id': 'submission_count'})

In [ ]:
successful_submissions = df[(df['is_final'] == 1) & (df['pre_score'] == 10000)].copy()

successful_assignment_count = successful_submissions.groupby('username')['assignment_id'].nunique().reset_index()
successful_assignment_count.rename(columns={'assignment_id': 'num_assignments_solved'}, inplace=True)

In [ ]:
problem_count = df.groupby('username', as_index=False)['problem_id'].nunique()
problem_count = problem_count.rename(columns={'problem_id': 'problem_count'})

In [ ]:
solved = (df['is_final'] == 1) & (df['pre_score'] == 10000)
problem_successful = df[solved].groupby('username', as_index=False)['problem_id'].nunique()
problem_successful = problem_successful.rename(columns={'problem_id': 'problem_successful'})

In [ ]:
half_solved = (df['is_final'] == 1) & (df['pre_score'] < 10000)
problem_not_successful = df[half_solved].groupby('username', as_index=False)['problem_id'].nunique()
problem_not_successful = problem_not_successful.rename(columns={'problem_id': 'problem_not_successful'})

In [ ]:
lated = (df['is_final'] == 1) & (df['coefficient'] < 100)
problem_late = df[lated].groupby('username', as_index=False)['problem_id'].nunique()
problem_late = problem_late.rename(columns={'problem_id': 'problem_late'})
problem_late['problem_late'].fillna(0, inplace=True)

/tmp/ipython-input-12-1083889013.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  problem_late['problem_late'].fillna(0, inplace=True)


In [ ]:
df['created_at'] = pd.to_datetime('2000-' + df['created_at'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['day'] = df['created_at'].dt.date

days_active = df.groupby('username', as_index=False)['day'].nunique()

In [ ]:
first_sub = df.groupby('username').head(1)[ ['username', 'created_at'] ]
first_sub = first_sub.rename(columns={'created_at': 'first_sub_dt'})

last_sub = df.groupby('username').tail(1)[ ['username', 'created_at'] ]
last_sub = last_sub.rename(columns={'created_at': 'last_sub_dt'})

time_range = first_sub.merge(last_sub, on='username')

time_range['active_range_days'] = (time_range['last_sub_dt'] - time_range['first_sub_dt']).dt.days
time_range.loc[time_range ['active_range_days'] < 0, 'active_range_days'] += 365

In [ ]:
first_1 = df.groupby(['username', 'assignment_id', 'problem_id']).head(1)[ ['username', 'assignment_id', 'problem_id', 'created_at'] ]
first_1 = first_1.rename(columns={'created_at': 'first_submission'})

last_1 = df.loc[(df['is_final']==1) & (df['pre_score'] == 10000)].groupby(['username',  'assignment_id', 'problem_id']).tail(1)[ ['username', 'assignment_id', 'problem_id', 'created_at'] ]
last_1 = last_1.rename(columns={'created_at': 'last_submission'})

avg_completion_time = pd.merge(first_1, last_1, on=['username', 'assignment_id', 'problem_id'], how='inner')
avg_completion_time['completion_time'] = (avg_completion_time['last_submission'] - avg_completion_time['first_submission']).dt.total_seconds()/(60*60*24)
avg_completion_time.loc[avg_completion_time['completion_time'] < 0, 'completion_time'] += 365

avg_completion_time = avg_completion_time[avg_completion_time['completion_time'] > 0].groupby(['username'], as_index=False)['completion_time'].mean()

avg_completion_time.rename(columns={'completion_time': 'avg_completion_time'}, inplace=True)

In [ ]:
first_submission = df.groupby(['username', 'assignment_id', 'problem_id']).head(1).copy()

successful_first_submission = first_submission[(first_submission['is_final'] == 1) & (first_submission['pre_score'] == 10000)].copy()

successful_first_submission_count = successful_first_submission.groupby('username')['problem_id'].count().reset_index()
successful_first_submission_count.rename(columns={'problem_id': 'successful_first_submission_count'}, inplace=True)

In [ ]:
submissions_per_problem_per_student = df.groupby(['username', 'problem_id']).size().reset_index(name='num_submissions')

avg_submissions_per_problem = submissions_per_problem_per_student.groupby('problem_id')['num_submissions'].mean().reset_index()
avg_submissions_per_problem.rename(columns={'num_submissions': 'avg_submissions_per_problem'}, inplace=True)

In [ ]:
df_sorted = df.sort_values(by=['username', 'assignment_id', 'problem_id', 'created_at'])

df_sorted['time_diff'] = df_sorted.groupby(['username', 'assignment_id', 'problem_id'])['created_at'].diff()
df_sorted['time_diff_seconds'] = df_sorted['time_diff'].dt.total_seconds()/(60*60*24)

avg_time_diff_per_problem = df_sorted.groupby(['username', 'assignment_id', 'problem_id'])['time_diff_seconds'].mean().reset_index()
avg_time_diff_per_assignment = avg_time_diff_per_problem.groupby(['username', 'assignment_id'])['time_diff_seconds'].mean().reset_index()

avg_time_diff_per_assignment.rename(columns={'time_diff_seconds': 'avg_time_diff_in_assignment'}, inplace=True)
avg_time_diff_overall = avg_time_diff_per_assignment.groupby('username')['avg_time_diff_in_assignment'].mean().reset_index()

avg_time_diff_overall.rename(columns={'avg_time_diff_in_assignment': 'avg_inter_submission_time_per_assignment'}, inplace=True)

In [ ]:
first_submission = df.groupby(['username', 'assignment_id', 'problem_id']).head(1).copy()

successful_first_submission = first_submission[(first_submission['is_final'] == 1) & (first_submission['pre_score'] == 10000)].copy()

successful_first_submission_count = successful_first_submission.groupby('username')['problem_id'].count().reset_index()
successful_first_submission_count.rename(columns={'problem_id': 'successful_first_submission_count'}, inplace=True)


In [ ]:
successful_students_per_problem = df[
    (df['is_final'] == 1) & (df['pre_score'] == 10000)
].groupby('problem_id')['username'].nunique().reset_index()
successful_students_per_problem.rename(columns={'username': 'num_successful_students'}, inplace=True)

total_students_per_problem = df.groupby('problem_id')['username'].nunique().reset_index()
total_students_per_problem.rename(columns={'username': 'num_total_students'}, inplace=True)

problem_success_rate = pd.merge(successful_students_per_problem, total_students_per_problem, on='problem_id', how='left')

problem_success_rate['student_success_rate'] = (problem_success_rate['num_successful_students'] / problem_success_rate['num_total_students']) * 100

problem_success_rate['student_success_rate'] = problem_success_rate['student_success_rate'].fillna(0)

In [ ]:
submissions_per_problem_per_student = df.groupby(['username', 'problem_id']).size().reset_index(name='num_submissions')

avg_submissions_per_problem = submissions_per_problem_per_student.groupby('problem_id')['num_submissions'].mean().reset_index()
avg_submissions_per_problem.rename(columns={'num_submissions': 'avg_submissions_per_problem'}, inplace=True)

In [ ]:
problem_difficulty = pd.merge(problem_success_rate[['problem_id', 'student_success_rate']], avg_submissions_per_problem, on='problem_id', how='left')

problem_difficulty['avg_submissions_per_problem'].fillna(0, inplace=True)

success_rate_threshold = 30
avg_submissions_threshold = 3.5

hard_problems = problem_difficulty[
    (problem_difficulty['student_success_rate'] < success_rate_threshold) &
    (problem_difficulty['avg_submissions_per_problem'] > avg_submissions_threshold)
]['problem_id'].tolist()

print("Các problem được coi là khó:", hard_problems)

Các problem được coi là khó: ['04681b16ae128da4be6bcdf7ae98b4c294ed20b5', '1caa82b6f5dd4d3821e16eb5bab47065c2d3fdd2', '28770d748d2e1d0e61280452505f938b32e3027e', '564a08c59baf99960947f1d01712a90758339043', '6ed29be1551f4966abbe6046a264ea2aa6d60302', '8c1495329b9727d0ec90edc8c9f0ac71f9785f9d', 'cc84d5fd74bf68bf6a2f7394c191b00a21167d49', 'ea4bf1c5098a2f435cd8b6ae3e3186678f641a63', 'ee79b6ec74701ad76dc37a2ad254231d9ca95ddc']


/tmp/ipython-input-23-805274638.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  problem_difficulty['avg_submissions_per_problem'].fillna(0, inplace=True)


In [ ]:
successful_submissions = df[(df['is_final'] == 1) & (df['pre_score'] == 10000)].copy()

successful_hard_problem_submissions = successful_submissions[
    successful_submissions['problem_id'].isin(hard_problems)
]

num_hard_problems_solved_per_student = successful_hard_problem_submissions.groupby('username')['problem_id'].nunique().reset_index()
num_hard_problems_solved_per_student.rename(columns={'problem_id': 'num_hard_problems_solved'}, inplace=True)


In [ ]:
unique_usernames_array = df['username'].unique()
data = pd.DataFrame(unique_usernames_array, columns=['username'])

data = pd.merge(data, total_per_user, on='username', how='left')
data = pd.merge(data, assignment_count, on='username', how='left')
data = pd.merge(data, successful_assignment_count, on='username', how='left')
data = pd.merge(data, submission_count, on='username', how='left')
data = pd.merge(data, problem_count, on='username', how='left')
data = pd.merge(data, problem_successful, on='username', how='left')
data = pd.merge(data, problem_not_successful, on='username', how='left')
data = pd.merge(data, problem_late, on='username', how='left')
data = pd.merge(data, days_active, on='username', how='left')
data = pd.merge(data, time_range[['username','active_range_days']], on='username', how='left')
data = pd.merge(data, avg_completion_time, on='username', how='left')
data = pd.merge(data, avg_time_diff_overall, on='username', how='left')
data = pd.merge(data, successful_first_submission_count, on='username', how='left')
data = pd.merge(data, num_hard_problems_solved_per_student, on='username', how='left')

In [ ]:
data['total_coefficient'] = data['total_coefficient'].fillna(0)
data['num_assignments_solved'] = data['num_assignments_solved'].fillna(0)
data['problem_successful'] = data['problem_successful'].fillna(0)
data['problem_not_successful'] = data['problem_not_successful'].fillna(0)
data['problem_late'] = data['problem_late'].fillna(0)
data['avg_completion_time'] = data['avg_completion_time'].fillna(0)
data['avg_inter_submission_time_per_assignment'] = data['avg_inter_submission_time_per_assignment'].fillna(0)
data['successful_first_submission_count'] = data['successful_first_submission_count'].fillna(0)
data['num_hard_problems_solved'] = data['num_hard_problems_solved'].fillna(0)

In [ ]:
data['percent_solved'] = (data['problem_successful'] / data['problem_count']) * 100

In [ ]:
data['percent_submit'] = (data['problem_successful'] / data['submission_count']) * 100

In [ ]:
data['score_per_problem'] = data['total_coefficient'] / data['problem_count']

In [ ]:
data['percent_active'] = 0

data.loc[data['active_range_days'] > 0, 'percent_active'] = (data.loc[data['active_range_days'] > 0, 'day'] / data.loc[data['active_range_days'] > 0, 'active_range_days']) * 100


/tmp/ipython-input-30-1207481186.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[14.28571429 31.03448276 17.14285714 ... 15.7480315  20.75471698
 29.09090909]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[data['active_range_days'] > 0, 'percent_active'] = (data.loc[data['active_range_days'] > 0, 'day'] / data.loc[data['active_range_days'] > 0, 'active_range_days']) * 100


In [ ]:
data['percent_first_submit_success'] = (data['successful_first_submission_count'] / data['problem_successful']) * 100
data['percent_first_submit_success'] = data['percent_first_submit_success'].fillna(0)

In [ ]:
data['total_coefficient_per_day'] = data['total_coefficient'] / data['day']
data['total_coefficient_per_day'] = data['total_coefficient_per_day'].fillna(0)


In [ ]:
df_scores = pd.read_csv('/gdrive/MyDrive/Project/Bai2/qt-public.csv')

In [ ]:
df_scores = df_scores.rename(columns={
    "hash": "username"
})

In [ ]:
data_final = pd.merge(data, df_scores, on='username', how='left')

In [ ]:
df_test = data_final[data_final['diemqt'].isna()]

df_train = data_final[data_final['diemqt'].notna()]

In [ ]:
df_train['diemqt'] = df_train['diemqt'].replace(r'^\s*$', 0, regex=True)
df_train['diemqt'] = df_train['diemqt'].astype(float)

/tmp/ipython-input-37-1220815538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['diemqt'] = df_train['diemqt'].replace(r'^\s*$', 0, regex=True)
/tmp/ipython-input-37-1220815538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['diemqt'] = df_train['diemqt'].astype(float)


In [ ]:
y_train = df_train['diemqt']

X_train = df_train.drop(columns=['username', 'diemqt'])

X_test = df_test.drop(columns=['username', 'diemqt'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
#273072
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor(random_state=42, max_depth=10, min_samples_split=10, n_estimators=100)
model.fit(X_train_split, y_train_split)

# Đánh giá trên tập validation
y_pred_val = model.predict(X_val_split)
r2 = r2_score(y_val_split, y_pred_val)
print(f"R^2 score trên tập validation: {r2}")

R^2 score trên tập validation: 0.15518826673419606


In [ ]:
#273074
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_split, y_train_split)
print("Best parameters found: ", grid_search.best_params_)
print("Best R^2 score found: ", grid_search.best_score_)

Best parameters found:  {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Best R^2 score found:  0.060147566076869705


In [ ]:
y_pred_val_1 = grid_search.predict(X_val_split)
r2 = r2_score(y_val_split, y_pred_val_1)
print(f"R^2 score trên tập validation: {r2}")

R^2 score trên tập validation: 0.1737413079396538


In [ ]:
predictions = grid_search.predict(X_test)

df_test['diemqt'] = predictions

df_test

/tmp/ipython-input-49-3576848380.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['diemqt'] = predictions


,username,total_coefficient,num_assignments_submitted,num_assignments_solved,submission_count,problem_count,problem_successful,problem_not_successful,problem_late,day,...,avg_inter_submission_time_per_assignment,successful_first_submission_count,num_hard_problems_solved,percent_solved,percent_submit,score_per_problem,percent_active,percent_first_submit_success,total_coefficient_per_day,diemqt
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,56850000.0,5,5.0,139,58,56.0,2.0,0.0,16,...,0.036895,30.0,0.0,96.551724,40.287770,9.801724e+05,14.285714,53.571429,3.553125e+06,8.003929
1,ba12c0a2cb367af0467e479c03507c71a805d291,117000000.0,6,6.0,319,112,112.0,0.0,0.0,36,...,5.618592,58.0,0.0,100.000000,35.109718,1.044643e+06,31.034483,51.785714,3.250000e+06,8.658890
3,b7298b0fe50443a623af9b56792b330c2d052845,109030700.0,5,5.0,247,110,108.0,2.0,0.0,29,...,7.925966,62.0,0.0,98.181818,43.724696,9.911882e+05,23.200000,57.407407,3.759679e+06,8.519961
4,c60be70309789b39355dc612f36e37090ccad5dc,80700000.0,10,9.0,172,83,76.0,7.0,0.0,23,...,0.099128,44.0,0.0,91.566265,44.186047,9.722892e+05,21.698113,57.894737,3.508696e+06,8.727824
6,a22a58c5be8aa2c2700619e37f2b7a6e4efa7e6b,100422200.0,11,11.0,288,103,98.0,5.0,0.0,38,...,0.274017,51.0,0.0,95.145631,34.027778,9.749728e+05,35.185185,52.040816,2.642689e+06,9.305354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,508e0627871ed482bd8ab34544e43e5d856a889c,33000000.0,4,4.0,94,33,33.0,0.0,0.0,10,...,1.052288,15.0,0.0,100.000000,35.106383,1.000000e+06,26.315789,45.454545,3.300000e+06,6.711842
1483,d525d6edf4b0c6362ddaa4814c3930a6a62bdc99,97600000.0,10,10.0,217,98,97.0,1.0,0.0,24,...,0.716111,64.0,0.0,98.979592,44.700461,9.959184e+05,21.818182,65.979381,4.066667e+06,8.524780
1484,b722e6209f2858faf0bf80947cadcbde586bb666,23788800.0,3,3.0,136,26,22.0,3.0,0.0,10,...,0.184549,6.0,0.0,84.615385,16.176471,9.149538e+05,22.222222,27.272727,2.378880e+06,6.659132
1486,b45e8d507778dab56f381a681c453cbf4d3b4050,46700000.0,10,8.0,141,53,45.0,9.0,0.0,20,...,0.103256,23.0,0.0,84.905660,31.914894,8.811321e+05,15.748031,51.111111,2.335000e+06,8.203376


In [ ]:
df_test[['username', 'diemqt']].to_csv('predicted_scores_2.csv', index=False)